In [1]:
import numpy as np
import sklearn 
import tensorflow as tf
from tensorflow import keras 

from keras import Model

from keras.optimizers import Adam, SGD
from keras.metrics import categorical_crossentropy
from keras.preprocessing.image import ImageDataGenerator 
from keras.layers import Dense

import itertools
import os, shutil, random, glob

from sklearn.metrics import confusion_matrix 

import matplotlib.pyplot as plt 
%matplotlib inline 

## **Image preparation**

Dataset: kaggle link
- 10 classes (digits 0-9)
- class data:
    - class 0:
    - class 1:
    - class 2:
    ...
    - class 10

## **Fine-tune MobileNet model**

In [2]:
train_path = "./data/train"
valid_path = "./data/valid"
test_path = "./data/test"

In [3]:
train_batches = ImageDataGenerator(preprocessing_function=keras.applications.mobilenet.preprocess_input).\
                                   flow_from_directory(train_path, target_size=(224,224), batch_size=80)

valid_batches = ImageDataGenerator(preprocessing_function=keras.applications.mobilenet.preprocess_input).\
                                   flow_from_directory(valid_path, target_size=(224,224), batch_size=35)
                                   
test_batches = ImageDataGenerator(preprocessing_function=keras.applications.mobilenet.preprocess_input).\
                                   flow_from_directory(test_path, target_size=(224,224), batch_size=35, shuffle=False)


Found 560 images belonging to 10 classes.
Found 70 images belonging to 10 classes.
Found 70 images belonging to 10 classes.


In [4]:
mobile = keras.applications.mobilenet.MobileNet()

2022-10-06 01:02:26.301125: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudnn.so.8'; dlerror: libcudnn.so.8: cannot open shared object file: No such file or directory
2022-10-06 01:02:26.301146: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1850] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
2022-10-06 01:02:26.301438: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [5]:
mobile.summary()

Model: "mobilenet_1.00_224"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 conv1 (Conv2D)              (None, 112, 112, 32)      864       
                                                                 
 conv1_bn (BatchNormalizatio  (None, 112, 112, 32)     128       
 n)                                                              
                                                                 
 conv1_relu (ReLU)           (None, 112, 112, 32)      0         
                                                                 
 conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)     288       
                                                                 
 conv_dw_1_bn (BatchNormaliz  (None, 112, 112, 32)     128       
 ation)                                         

In [6]:
x = mobile.layers[-2].output  # exclude last 5 layers (can vary) -4

In [7]:
predictions = Dense(10, activation="softmax", name="predictions")(x)

In [8]:
model = Model(inputs = mobile.input, outputs=predictions)

In [9]:
model.summary() 

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 conv1 (Conv2D)              (None, 112, 112, 32)      864       
                                                                 
 conv1_bn (BatchNormalizatio  (None, 112, 112, 32)     128       
 n)                                                              
                                                                 
 conv1_relu (ReLU)           (None, 112, 112, 32)      0         
                                                                 
 conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)     288       
                                                                 
 conv_dw_1_bn (BatchNormaliz  (None, 112, 112, 32)     128       
 ation)                                                      

In [10]:
# freeze weights of all layers except the last 24 layers because
# hand sign language i.e classes are different than in ImageNet library

for layer in model.layers[:-28]: # tinker with this number 
    layer.trainable = False

## **Train the model**

In [11]:
model.compile(Adam(learning_rate=0.0001), loss="categorical_crossentropy", metrics=["accuracy"])

In [12]:
model.fit(train_batches, steps_per_epoch=7, validation_data=valid_batches, \
                        validation_steps=2, epochs=20, verbose=2)

Epoch 1/20
7/7 - 9s - loss: 2.0334 - accuracy: 0.5036 - val_loss: 4.6329 - val_accuracy: 0.3000 - 9s/epoch - 1s/step
Epoch 2/20
7/7 - 7s - loss: 0.1177 - accuracy: 0.9768 - val_loss: 4.2509 - val_accuracy: 0.3429 - 7s/epoch - 972ms/step
Epoch 3/20
7/7 - 7s - loss: 0.0335 - accuracy: 0.9946 - val_loss: 4.0190 - val_accuracy: 0.4429 - 7s/epoch - 983ms/step
Epoch 4/20
7/7 - 7s - loss: 0.0030 - accuracy: 1.0000 - val_loss: 3.8580 - val_accuracy: 0.5143 - 7s/epoch - 945ms/step
Epoch 5/20
7/7 - 7s - loss: 0.0041 - accuracy: 1.0000 - val_loss: 3.5792 - val_accuracy: 0.5571 - 7s/epoch - 984ms/step
Epoch 6/20
7/7 - 7s - loss: 0.0014 - accuracy: 1.0000 - val_loss: 3.2596 - val_accuracy: 0.6000 - 7s/epoch - 1s/step
Epoch 7/20
7/7 - 7s - loss: 9.1643e-04 - accuracy: 1.0000 - val_loss: 2.9224 - val_accuracy: 0.6286 - 7s/epoch - 1000ms/step
Epoch 8/20
7/7 - 7s - loss: 9.6256e-04 - accuracy: 1.0000 - val_loss: 2.5911 - val_accuracy: 0.6571 - 7s/epoch - 1s/step
Epoch 9/20
7/7 - 7s - loss: 5.5950e-04 -

In [ ]:
# try training for longer or adjusting hyper parameters to see accuracy in performance 

In [13]:
model.save('mobilenet')

INFO:tensorflow:Assets written to: mobilenet/assets


INFO:tensorflow:Assets written to: mobilenet/assets


## **Predict sign language digits**

In [14]:
test_labels = test_batches.classes
test_labels

array([0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 3,
       3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 5, 6, 6,
       6, 6, 6, 6, 6, 7, 7, 7, 7, 7, 7, 7, 8, 8, 8, 8, 8, 8, 8, 9, 9, 9,
       9, 9, 9, 9], dtype=int32)

In [15]:
predictions = model.predict_generator(test_batches, steps=2, verbose=2) #0

/tmp/ipykernel_1379154/2824382785.py:1: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  predictions = model.predict_generator(test_batches, steps=2, verbose=2) #0


2/2 - 1s - 757ms/epoch - 378ms/step


In [16]:
cm = confusion_matrix(test_labels, predictions.argmax(axis=1))
cm

array([[7, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 7, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 7, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 7, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 6, 1, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 7, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 1, 6, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 7, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 7, 0],
       [0, 0, 0, 0, 0, 1, 0, 0, 0, 6]])

In [17]:
test_batches.class_indices

{'0': 0,
 '1': 1,
 '2': 2,
 '3': 3,
 '4': 4,
 '5': 5,
 '6': 6,
 '7': 7,
 '8': 8,
 '9': 9}

In [ ]:
# cm_plot_labels = []
# plot_confusion_matrix(cm, cm_plot_labels, title="Confusion Matrix")